In [ ]:
# 손실함수(Loss function)와 비용함수(cost function)의 차이
# 보통 동일한 의미로 사용되지만 엄밀히 구분하자면,

# 손실함수(Loss function)는 샘플 하나에 대한 손실을 의미하며

# 비용함수(cost function)는 훈련세트에 있는 모든 샘플에 대한 손실함수의 합을 의미한다.


In [ ]:
# 순전파 : 기울기(gradient)를 사용하는 것이 아닌, 각 층에서의 가중치와 활성화 함수에 의해 입력 데이터가 변환되는 과정을 수행합니다. 가중치는  다양한 데이터에 적응할 수 있도록 무작위 초기화  sigmoid : 0~0.25, tanh:-1~1
# 역전파 : 손실 함수의 기울기(gradient)를 계산하고, 이를 사용하여 모델의 가중치를 업데이트하는 과정을 말합니다. 손실 함수를 통해 모델의 예측과 실제 타깃 값 사이의 차이를 계산합니다. (최적화 알고리즘 사용 x)
# 각 층의 가중치에 대한 손실 함수의 기울기를 계산하고, 이를 이용하여 경사하강법 등의 최적화 알고리즘을 사용하여 가중치를 업데이트합니다. 역전파 과정에서는 기울기를 미분하고 연쇄 법칙을 이용하여 값들을 계산합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#연습문제
#1. 전처리(불용어 제거)
#2. 모델 구조 변경 및 성능 개선
#3. 소설 책(토) 중 일부를 발췌하여 모델링 연습 (선택사항, 형태소분석기?)

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from string import punctuation

In [ ]:
texts=['자연어 처리 알고리즘', '자연어 처리 방법',
       '자연어 NLP 알고리즘 알고리즘',
       '자연어 처리 전문가']

In [ ]:
tok=Tokenizer()#클래스(설계도), 객체(건물)
tok.fit_on_texts(texts)

In [ ]:
tok.index_word  # index_word 가 1부터 시작되지만 matrix에서는 0부터 시작

In [ ]:
tok.texts_to_matrix(texts) # default : mode='binary'
tok.texts_to_matrix(texts, mode='binary')
tok.texts_to_matrix(texts, mode='count')
tok.texts_to_matrix(texts, mode='tfidf')
tok.texts_to_matrix(texts, mode='freq')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/dataset/NYT_2018.csv")
df
df.headline.isnull().sum()

In [ ]:
headline=[]
headline.extend(list(df.headline.values))  # 비슷하지만 append는 원소 1개/ extend는 iteratble 모든 항목을 추가한다.

In [ ]:
headline

In [ ]:
len(headline) #1324

In [ ]:
sum(df.headline=='Unknown')

In [ ]:
len([w for w in headline if w=='Unknown'])

In [ ]:
headline=[w for w in headline if w!='Unknown']
len(headline)

In [ ]:
# headline에 저장된 뉴스 기사 제목으로 다음 단어를 생성하는 LSTM
# 기반 모델 설계
# 동작 예
# 입력 : I, 생성하고자 하는 단어의 갯수
# 출력 : I was ...완성

In [ ]:
def pre_func(title):
    #소문자 변환
    res=''.join(w.lower() for w in title if w not in punctuation)
    return res

In [ ]:
punctuation

In [ ]:
pre_headline=[pre_func(x) for x in headline]
"""
동작순서
x in headline -> pre_func(x) -> pre_func(title) -> for w in title
->if w not in punctuation -> w.lower() -> ''.join -> 소문자로 변환된 단어들이 연결됨
-> return res -> pre_headline리스트의 요소로 저장됨
"""

In [ ]:
pre_headline

In [ ]:
tok=Tokenizer()

In [ ]:
tok.fit_on_texts(pre_headline)

In [ ]:
tok.index_word
len(tok.index_word)

In [ ]:
vocab_size=len(tok.index_word)+1 #3620

In [ ]:
pre_headline

In [ ]:
sequences=[]
for s in pre_headline:
    print(tok.texts_to_sequences([s])[0]) #각 문장별 인코딩


In [ ]:
sequences=[]
for s in pre_headline:
    # print(s)
    print(tok.texts_to_sequences([s])[0]) #각 문장별 인코딩      2차원 배열내의 정보 가져오기 위해 [0]
    enc=tok.texts_to_sequences([s])[0]
    # 질문!! 반복문이 i가 다시 0부터 할당된느거아님?



    for i in range(1, len(enc)):
        seq=enc[:i+1]
        sequences.append(seq)

In [ ]:
sequences[:15]

# ex)lstm 모델로 단어를 예측하도록 설계합니다.
# 입력                                    출력
# ------------------------------------------------------
# lstm                                    모델로
# lstm 모델로                             단어를
# ...
# lstm 모델로 단어를 예측하도록           설계합니다




In [ ]:
# key, value 교환
idx2word={}
type(tok.word_index)
for k, v in tok.word_index.items():
    idx2word[v]=k

In [ ]:
idx2word

In [ ]:
print(max(len(i) for i in sequences)) #24단어 문장이 가장 길다
ml=max(len(i) for i in sequences)

In [ ]:
#[len(i) for i in sequences]

In [ ]:
sequences[1]

In [ ]:
sequences=pad_sequences(sequences, maxlen=ml, padding='pre')

In [ ]:
sequences[1]

In [ ]:
sequences[8]

In [ ]:
#sequences를 x,y로 분리하여 저장
# x = [i[:-1] for i in sequences ]
# y = [i[-1] for i in sequences ]
sequences=np.array(sequences)
x=sequences[:,:-1]
y=sequences[:,-1]

In [ ]:
x.shape
y.shape

In [ ]:
sequences.shape #(7809, 24)

In [ ]:
y

In [ ]:
y=to_categorical(y, vocab_size) # 원핫 인코딩  vocab_size : 인코딩 개수

In [ ]:
y.shape

In [ ]:
len(sequences[0])

In [ ]:
# Embedding() : Embedding()은 단어를 밀집 벡터로 만드는 역할을 합니다.
# 단어 임베딩은 차원을 효과적으로 압축하여 희소성을 줄이고, 단어 간의 의미적 유사성을 파악하기 위해 많이 사용되는 기술입니다.
model=Sequential()
model.add(Embedding(vocab_size, 10))#3620차원을 -> 10차원으로 임베딩
model.add(LSTM(128,return_sequences=True))  # return_sequences=true
model.add(LSTM(64,return_sequences=False))   # 마지막 LSTM모델에서는  return_sequences=false
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x,y,epochs=200, verbose=2)

In [ ]:
# 입력 -> '나는' + 단어 10개 예상

# 나는 -> 모델 -> 나는 지금 -> 모델 -> 나는 지금 집에 -> 모델 ->...
# ->나는 지금 집에 있는데 TV를 시청하고 있다...


In [ ]:
def gen_sent(model, tok, c_word, n):  #tok : 인코딩해놓은 단어사전
    pred_sent=''
    for _ in range(n):
      enc=tok.texts_to_sequences([c_word])[0]  # c_world 에 해당하는 인덱싱 배열
      enc=pad_sequences([enc], maxlen=ml, padding='pre')  #pad_sequences 는 항상 2차원 배열을 입력받음
      res=model.predict(enc)  #enc:리스트형태
      res=np.argmax(res)
      for w, i in tok.word_index.items():
          if i==res:
              break
      print("예측단어:",w)
      c_word=c_word+ " " + w    #  The=>  The new
    pred_sent=c_word
    return pred_sent



In [ ]:
enc

In [ ]:
gen_sent(model, tok, 'The', 10)

In [ ]:
# 참고사이트
# word2vec.kr